# Analyze and Visualize Results

Used during the hyperparameter search to analyze multiple runs and compare performance across them.

In [10]:
import sys, os, logging
import json
from pathlib import Path
from typing import List, Tuple, Union, Dict, Optional
from multiprocessing import Process, Pool
import psutil
import base64
import asyncio
import cv2
import imageio as iio
import numpy as np
from IPython.core.display import HTML
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq, fftn, fftshift
from pprint import pprint

from turbx import REPO_PATH, log
from turbx.vis import (calc_tfpnr, calc_tdr, calc_frames_removed, label_to_per_frame_list)
from turbx.metrics import (
    calc_box_area,
    boxes_to_binary,
    target_detection_rate,
    tfpnr,
    safe_division
)

## Load experiment output

In [11]:
base_path = Path(f"{REPO_PATH}/notebooks/outputs/ablation/2022-12-23/22-38-06")

## Calculate metrics on experiment data

In [12]:
# calculate per video scores

# Things we want per run,
# 1) mAR
# 2) mAP
# 3) F1
# 4) TDR - if this is not 1 then we can adjust this param search
# TDR can be tought of as the notification rate, we really do not want to miss a notification

all_neg_dets= 0
all_neg_lab= 0
all_frames =0


maxF1 = 0
maxAP = 0
maxAR = 0
maxFramesRemoved = 0
maxTDR = 0
APs = []
ARs = []
F1s = []
FRs = []
TDRs = []
satisfies_criterion_param_set = []
satisfies_criterion = []
maxAVGFRTDRParamID = None
maxAVGFRTDRIDX = None
maxAVGFRTDR = 0

args_json = Path(base_path / "args.json")
if args_json.exists():
    with open(str(args_json), 'r') as f:
        pprint(json.load(f))

#print("Per frame:")
#print("id:, FPR, FNR, Target Detection Rate, % Frames Removed, % Neg Frames Removed")

# accumulate run directories
subdirs = next(os.walk(str(base_path)))[1]
# iterate through each run, and calculate stats
for i, param_set in enumerate(subdirs):
    param_set_path = Path(base_path/param_set)
    param_set_results = []
    # load every json for that run and append to param_set_results
    for f_name in param_set_path.glob("**/*.json"):
        with open(f_name, 'r') as f:
            params = json.load(f)
            param_set_results.append((params, param_set))
    tp = 0
    fp = 0
    fn = 0
    all_neg_dets = 0
    all_neg_lab = 0
    all_frames = 0
    tdrsum=0

    # for each video
    for result, param_set_id in param_set_results:
        # get per video statistics
        binary_label, binary_pred, tfpnr_dict = calc_tfpnr(result['label'], result['prediction'], show=False, save=False)
        unique_targs, det_targs, tdr = calc_tdr(result['label'], result['prediction'])
        _, n_pos_dets, n_neg_dets = calc_frames_removed(result['prediction'])
        _, n_pos_lab, n_neg_lab = calc_frames_removed(label_to_per_frame_list(result['label']))
        perc_frames_removed = 100*(n_neg_dets / len(binary_pred))
        perc_neg_frame_removed = 100*(n_neg_dets / n_neg_lab)
        # TODO: weight videos equally, or weight by frame? -> weight by frame
        # Blerim is adding up all then dividing later -> fair, is by frame
        all_neg_dets += n_neg_dets
        all_neg_lab += n_neg_lab
        all_frames += len(binary_pred)
        tp += tfpnr_dict['tp']
        fp += tfpnr_dict['fp']
        fn += tfpnr_dict['fn']
        tdrsum += tdr
    
    param_set_AP = tp/(tp+fp)
    param_set_AR = tp/(tp+fn)
    #param_set_AP = safe_division(tp, tp+fp, 1.0)
    #param_set_AR = safe_division(tp, tp+fn, 1.0)
    param_set_F1 = 2/((1/param_set_AR) + (1/param_set_AP))
    param_set_tdr = tdrsum/len(param_set_results)
    APs.append(param_set_AP)
    ARs.append(param_set_AR)
    F1s.append(param_set_F1)
    TDRs.append(param_set_tdr)
    perc_all_frames = 100*(all_neg_dets/all_frames)
    perc_tn_frames = 100*(all_neg_lab/all_frames)
    FRs.append(perc_all_frames)

    if(param_set_AP > maxAP):
        maxAP = param_set_AP
        maxAPParamID = param_set
        maxAPParamIDX = i
    if(param_set_AR > maxAR):
        maxAR = param_set_AR
        maxARParamID = param_set
        maxARParamIDX = i
    if(param_set_F1 > maxF1):
        maxF1 = param_set_F1
        maxF1ParamID = param_set
        maxF1ParamIDX = i
    if(param_set_tdr > maxTDR):
        maxTDR = param_set_tdr
        maxTDRParamID = param_set
        maxTDRParamIDX = i
    
    # calculate best frame remover
    if(perc_all_frames > maxFramesRemoved):
        maxFramesRemoved = perc_all_frames
        maxFRParamID = param_set
        maxFRParamIDX = i

    if (param_set_tdr > .9) and (perc_all_frames > 20):
        fr = perc_all_frames/100.
        avg_tdr_fr = (1/2.)*(param_set_tdr + fr)
        satisfies_criterion_param_set.append(param_set)
        satisfies_criterion.append(i) 
        if avg_tdr_fr > maxAVGFRTDR:
            maxAVGFRTDRParamID = param_set
            maxAVGFRTDRIDX = i
            maxAVGFRTDR = avg_tdr_fr
    
    #print(f"\n mean false negative rate: {mean_fnr:.2f}")
    #print(f"\n% of all frames removed: {perc_all_frames:.2f}")
    #print(f"\n% of all frames that are empty: {perc_tn_frames:.2f}")
print(f"Best Average Precision:\nParam Set: {maxAPParamID},\tAP: {APs[maxAPParamIDX]:.2f},\tAR: {ARs[maxAPParamIDX]:.2f},\tF1: {F1s[maxAPParamIDX]:.2f},\tFR: {FRs[maxAPParamIDX]:.2f},\tTDR: {TDRs[maxAPParamIDX]:.2f}")
print(f"Best Average Recall:\nParam Set: {maxARParamID},\tAP: {APs[maxARParamIDX]:.2f},\tAR: {ARs[maxARParamIDX]:.2f},\tF1: {F1s[maxARParamIDX]:.2f},\tFR: {FRs[maxARParamIDX]:.2f},\tTDR: {TDRs[maxARParamIDX]:.2f}")
print(f"Best F1:\nParam Set: {maxF1ParamID},\tAP: {APs[maxF1ParamIDX]:.2f},\tAR: {ARs[maxF1ParamIDX]:.2f},\tF1: {F1s[maxF1ParamIDX]:.2f},\tFR: {FRs[maxF1ParamIDX]:.2f},\tTDR: {TDRs[maxF1ParamIDX]:.2f}")
print(f"Best Frame Remover:\nParam Set: {maxFRParamID},\tAP: {APs[maxFRParamIDX]:.2f},\tAR: {ARs[maxFRParamIDX]:.2f},\tF1: {F1s[maxFRParamIDX]:.2f},\tFR: {FRs[maxFRParamIDX]:.2f},\tTDR: {TDRs[maxFRParamIDX]:.2f}")
print(f"Best Target Detection Rate:\nParam Set: {maxTDRParamID},\tAP: {APs[maxTDRParamIDX]:.2f},\tAR: {ARs[maxTDRParamIDX]:.2f},\tF1: {F1s[maxTDRParamIDX]:.2f},\tFR: {FRs[maxTDRParamIDX]:.2f},\tTDR: {TDRs[maxTDRParamIDX]:.2f}")
if maxAVGFRTDRParamID is not None:
    print(f"\nBest FR and TDR mixture:\nParam Set: {maxAVGFRTDRParamID},\tAP: {APs[maxAVGFRTDRIDX]:.2f},\tAR: {ARs[maxAVGFRTDRIDX]:.2f},\tF1: {F1s[maxAVGFRTDRIDX]:.2f},\tFR: {FRs[maxAVGFRTDRIDX]:.2f},\tTDR: {TDRs[maxAVGFRTDRIDX]:.2f}\n")
    print("Satisfies criterion:")
    for params, idx in zip(satisfies_criterion_param_set, satisfies_criterion):
        print(f"Param Set: {params},\tAP: {APs[idx]:.2f},\tAR: {ARs[idx]:.2f},\tF1: {F1s[idx]:.2f},\tFR: {FRs[idx]:.2f},\tTDR: {TDRs[idx]:.2f}")
else:
    print("No runs satisfy criterion!")
print("All Runs:")
for idx, param_set in enumerate(subdirs):
    print(f"Param Set: {param_set},\tAP: {APs[idx]:.2f},\tAR: {ARs[idx]:.2f},\tF1: {F1s[idx]:.2f},\tFR: {FRs[idx]:.2f},\tTDR: {TDRs[idx]:.2f}")

{'dataset': 'test',
 'filters': ['turbine_filter',
             'denoise_filter',
             'intensity_filter',
             'tracklet_association'],
 'params': ['experiments/20.json',
            'experiments/88.json',
            'experiments/75.json'],
 'path': '/home/nowa201/Projects/triton-crdp/turbx/data'}
Best Average Precision:
Param Set: 1,	AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.09,	TDR: 1.00
Best Average Recall:
Param Set: 0,	AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00
Best F1:
Param Set: 1,	AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.09,	TDR: 1.00
Best Frame Remover:
Param Set: 1,	AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.09,	TDR: 1.00
Best Target Detection Rate:
Param Set: 0,	AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00
No runs satisfy criterion!
All Runs:
Param Set: 0,	AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00
Param Set: 1,	AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.09,	TDR: 1.00
Param Set: 2,	AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00


# Results



## Training Set


### No tracklet (misaligned params) results


```
Best Average Precision:
Param Set: 187,	AP: 0.40,	AR: 0.34,	F1: 0.37,	FR: 86.60,	TDR: 0.68
Best Average Recall:
Param Set: 57,	AP: 0.20,	AR: 0.88,	F1: 0.33,	FR: 30.88,	TDR: 0.99
Best F1:
Param Set: 168,	AP: 0.35,	AR: 0.49,	F1: 0.41,	FR: 77.49,	TDR: 0.88
Best Frame Remover:
Param Set: 188,	AP: 0.40,	AR: 0.28,	F1: 0.33,	FR: 88.53,	TDR: 0.59
Best Target Detection Rate:
 Param Set: 57,	AP: 0.20,	AR: 0.88,	F1: 0.33,	FR: 30.88,	TDR: 0.99
```



### No tracklet (aligned params) results



```
Best Average Precision:
Param Set: 89,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Best Average Recall:
Param Set: 20,	AP: 0.18,	AR: 0.95,	F1: 0.31,	FR: 16.31,	TDR: 0.99
Best F1:
Param Set: 89,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Best Frame Remover:
Param Set: 89,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Best Target Detection Rate:
Param Set: 20,	AP: 0.18,	AR: 0.95,	F1: 0.31,	FR: 16.31,	TDR: 0.99

Best FR and TDR mixture:
Param Set: 89,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94

Satisfies criterion:
Param Set: 61,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 32.96,	TDR: 0.99
Param Set: 95,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.76,	TDR: 0.99
Param Set: 59,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 32.96,	TDR: 0.99
Param Set: 92,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.76,	TDR: 0.99
Param Set: 66,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.28,	TDR: 0.95
Param Set: 104,	AP: 0.28,	AR: 0.70,	F1: 0.40,	FR: 60.64,	TDR: 0.94
Param Set: 50,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.58,	TDR: 0.95
Param Set: 68,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.28,	TDR: 0.95
Param Set: 103,	AP: 0.28,	AR: 0.70,	F1: 0.40,	FR: 60.64,	TDR: 0.94
Param Set: 57,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 32.96,	TDR: 0.99
Param Set: 32,	AP: 0.20,	AR: 0.85,	F1: 0.32,	FR: 31.92,	TDR: 0.97
Param Set: 35,	AP: 0.20,	AR: 0.85,	F1: 0.32,	FR: 31.92,	TDR: 0.97
Param Set: 102,	AP: 0.28,	AR: 0.70,	F1: 0.40,	FR: 60.64,	TDR: 0.94
Param Set: 69,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.28,	TDR: 0.95
Param Set: 56,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 32.96,	TDR: 0.99
Param Set: 105,	AP: 0.28,	AR: 0.70,	F1: 0.40,	FR: 60.64,	TDR: 0.94
Param Set: 51,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.58,	TDR: 0.95
Param Set: 58,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 32.96,	TDR: 0.99
Param Set: 67,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.28,	TDR: 0.95
Param Set: 93,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.76,	TDR: 0.99
Param Set: 94,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.76,	TDR: 0.99
Param Set: 60,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 32.96,	TDR: 0.99
Param Set: 34,	AP: 0.20,	AR: 0.85,	F1: 0.32,	FR: 31.92,	TDR: 0.97
Param Set: 33,	AP: 0.20,	AR: 0.85,	F1: 0.32,	FR: 31.92,	TDR: 0.97
Param Set: 27,	AP: 0.20,	AR: 0.85,	F1: 0.32,	FR: 31.92,	TDR: 0.97
Param Set: 9,	AP: 0.20,	AR: 0.85,	F1: 0.33,	FR: 32.69,	TDR: 0.97
Param Set: 11,	AP: 0.20,	AR: 0.85,	F1: 0.33,	FR: 32.69,	TDR: 0.97
Param Set: 29,	AP: 0.20,	AR: 0.85,	F1: 0.32,	FR: 31.92,	TDR: 0.97
Param Set: 16,	AP: 0.20,	AR: 0.85,	F1: 0.33,	FR: 32.69,	TDR: 0.97
Param Set: 42,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 33.11,	TDR: 0.99
Param Set: 89,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Param Set: 45,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.58,	TDR: 0.95
Param Set: 73,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.91,	TDR: 0.99
Param Set: 87,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Param Set: 80,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.91,	TDR: 0.99
Param Set: 74,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.91,	TDR: 0.99
Param Set: 28,	AP: 0.20,	AR: 0.85,	F1: 0.32,	FR: 31.92,	TDR: 0.97
Param Set: 17,	AP: 0.20,	AR: 0.85,	F1: 0.33,	FR: 32.69,	TDR: 0.97
Param Set: 10,	AP: 0.20,	AR: 0.85,	F1: 0.33,	FR: 32.69,	TDR: 0.97
Param Set: 75,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.91,	TDR: 0.99
Param Set: 81,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Param Set: 86,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Param Set: 72,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.91,	TDR: 0.99
Param Set: 44,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 33.11,	TDR: 0.99
Param Set: 43,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 33.11,	TDR: 0.99
Param Set: 88,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Param Set: 38,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 33.11,	TDR: 0.99
Param Set: 36,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 33.11,	TDR: 0.99
Param Set: 31,	AP: 0.20,	AR: 0.85,	F1: 0.32,	FR: 31.92,	TDR: 0.97
Param Set: 91,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.76,	TDR: 0.99
Param Set: 65,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.28,	TDR: 0.95
Param Set: 62,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 32.96,	TDR: 0.99
Param Set: 96,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.76,	TDR: 0.99
Param Set: 100,	AP: 0.28,	AR: 0.70,	F1: 0.40,	FR: 60.64,	TDR: 0.94
Param Set: 54,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 32.96,	TDR: 0.99
Param Set: 107,	AP: 0.28,	AR: 0.70,	F1: 0.40,	FR: 60.64,	TDR: 0.94
Param Set: 98,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.76,	TDR: 0.99
Param Set: 53,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.58,	TDR: 0.95
Param Set: 30,	AP: 0.20,	AR: 0.85,	F1: 0.32,	FR: 31.92,	TDR: 0.97
Param Set: 37,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 33.11,	TDR: 0.99
Param Set: 39,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 33.11,	TDR: 0.99
Param Set: 106,	AP: 0.28,	AR: 0.70,	F1: 0.40,	FR: 60.64,	TDR: 0.94
Param Set: 99,	AP: 0.28,	AR: 0.70,	F1: 0.40,	FR: 60.64,	TDR: 0.94
Param Set: 52,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.58,	TDR: 0.95
Param Set: 101,	AP: 0.28,	AR: 0.70,	F1: 0.40,	FR: 60.64,	TDR: 0.94
Param Set: 55,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 32.96,	TDR: 0.99
Param Set: 97,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.76,	TDR: 0.99
Param Set: 63,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.28,	TDR: 0.95
Param Set: 64,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.28,	TDR: 0.95
Param Set: 90,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.76,	TDR: 0.99
Param Set: 46,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.58,	TDR: 0.95
Param Set: 79,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.91,	TDR: 0.99
Param Set: 41,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 33.11,	TDR: 0.99
Param Set: 83,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Param Set: 77,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.91,	TDR: 0.99
Param Set: 48,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.58,	TDR: 0.95
Param Set: 70,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.28,	TDR: 0.95
Param Set: 84,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Param Set: 15,	AP: 0.20,	AR: 0.85,	F1: 0.33,	FR: 32.69,	TDR: 0.97
Param Set: 12,	AP: 0.20,	AR: 0.85,	F1: 0.33,	FR: 32.69,	TDR: 0.97
Param Set: 85,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Param Set: 71,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.28,	TDR: 0.95
Param Set: 76,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.91,	TDR: 0.99
Param Set: 82,	AP: 0.29,	AR: 0.70,	F1: 0.41,	FR: 60.88,	TDR: 0.94
Param Set: 49,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.58,	TDR: 0.95
Param Set: 40,	AP: 0.22,	AR: 0.90,	F1: 0.35,	FR: 33.11,	TDR: 0.99
Param Set: 47,	AP: 0.24,	AR: 0.79,	F1: 0.37,	FR: 47.58,	TDR: 0.95
Param Set: 78,	AP: 0.25,	AR: 0.83,	F1: 0.38,	FR: 46.91,	TDR: 0.99
Param Set: 13,	AP: 0.20,	AR: 0.85,	F1: 0.33,	FR: 32.69,	TDR: 0.97
Param Set: 14,	AP: 0.20,	AR: 0.85,	F1: 0.33,	FR: 32.69,	TDR: 0.97
```



### With tracklet results



```
Best Average Precision:
Param Set: 88,	AP: 0.34,	AR: 0.63,	F1: 0.44,	FR: 70.04,	TDR: 0.74
Best Average Recall:
Param Set: 20,	AP: 0.19,	AR: 0.94,	F1: 0.32,	FR: 22.21,	TDR: 0.99
Best F1:
Param Set: 88,	AP: 0.34,	AR: 0.63,	F1: 0.44,	FR: 70.04,	TDR: 0.74
Best Frame Remover:
Param Set: 87,	AP: 0.34,	AR: 0.62,	F1: 0.44,	FR: 70.46,	TDR: 0.73
Best Target Detection Rate:
Param Set: 20,	AP: 0.19,	AR: 0.94,	F1: 0.32,	FR: 22.21,	TDR: 0.99

Best FR and TDR mixture:
Param Set: 75,	AP: 0.28,	AR: 0.78,	F1: 0.41,	FR: 55.75,	TDR: 0.92

Satisfies criterion:
Param Set: 61,	AP: 0.24,	AR: 0.89,	F1: 0.38,	FR: 40.48,	TDR: 0.99
Param Set: 95,	AP: 0.28,	AR: 0.79,	F1: 0.41,	FR: 54.01,	TDR: 0.92
Param Set: 59,	AP: 0.24,	AR: 0.89,	F1: 0.37,	FR: 39.74,	TDR: 0.99
Param Set: 92,	AP: 0.28,	AR: 0.79,	F1: 0.41,	FR: 54.01,	TDR: 0.92
Param Set: 57,	AP: 0.24,	AR: 0.87,	F1: 0.37,	FR: 40.95,	TDR: 0.94
Param Set: 56,	AP: 0.24,	AR: 0.89,	F1: 0.37,	FR: 39.74,	TDR: 0.99
Param Set: 58,	AP: 0.24,	AR: 0.89,	F1: 0.38,	FR: 40.48,	TDR: 0.99
Param Set: 93,	AP: 0.28,	AR: 0.78,	F1: 0.41,	FR: 55.50,	TDR: 0.92
Param Set: 94,	AP: 0.28,	AR: 0.79,	F1: 0.41,	FR: 54.65,	TDR: 0.92
Param Set: 60,	AP: 0.24,	AR: 0.87,	F1: 0.37,	FR: 40.95,	TDR: 0.94
Param Set: 20,	AP: 0.19,	AR: 0.94,	F1: 0.32,	FR: 22.21,	TDR: 0.99
Param Set: 18,	AP: 0.20,	AR: 0.94,	F1: 0.33,	FR: 23.33,	TDR: 0.99
Param Set: 0,	AP: 0.20,	AR: 0.94,	F1: 0.33,	FR: 24.37,	TDR: 0.99
Param Set: 7,	AP: 0.20,	AR: 0.94,	F1: 0.33,	FR: 23.58,	TDR: 0.99
Param Set: 42,	AP: 0.24,	AR: 0.87,	F1: 0.37,	FR: 41.35,	TDR: 0.94
Param Set: 73,	AP: 0.28,	AR: 0.79,	F1: 0.41,	FR: 54.80,	TDR: 0.92
Param Set: 80,	AP: 0.28,	AR: 0.80,	F1: 0.41,	FR: 54.11,	TDR: 0.92
Param Set: 74,	AP: 0.28,	AR: 0.80,	F1: 0.41,	FR: 54.11,	TDR: 0.92
Param Set: 6,	AP: 0.20,	AR: 0.94,	F1: 0.33,	FR: 24.37,	TDR: 0.99
Param Set: 1,	AP: 0.20,	AR: 0.94,	F1: 0.33,	FR: 23.58,	TDR: 0.99
Param Set: 19,	AP: 0.20,	AR: 0.94,	F1: 0.32,	FR: 22.61,	TDR: 0.99
Param Set: 26,	AP: 0.19,	AR: 0.94,	F1: 0.32,	FR: 22.21,	TDR: 0.99
Param Set: 8,	AP: 0.20,	AR: 0.94,	F1: 0.32,	FR: 23.03,	TDR: 0.99
Param Set: 21,	AP: 0.20,	AR: 0.94,	F1: 0.33,	FR: 23.33,	TDR: 0.99
Param Set: 75,	AP: 0.28,	AR: 0.78,	F1: 0.41,	FR: 55.75,	TDR: 0.92
Param Set: 72,	AP: 0.28,	AR: 0.78,	F1: 0.41,	FR: 55.75,	TDR: 0.92
Param Set: 44,	AP: 0.24,	AR: 0.88,	F1: 0.37,	FR: 40.08,	TDR: 0.98
Param Set: 43,	AP: 0.24,	AR: 0.88,	F1: 0.38,	FR: 40.88,	TDR: 0.98
Param Set: 38,	AP: 0.24,	AR: 0.88,	F1: 0.37,	FR: 40.08,	TDR: 0.98
Param Set: 36,	AP: 0.24,	AR: 0.87,	F1: 0.37,	FR: 41.35,	TDR: 0.94
Param Set: 91,	AP: 0.28,	AR: 0.79,	F1: 0.41,	FR: 54.65,	TDR: 0.92
Param Set: 62,	AP: 0.24,	AR: 0.89,	F1: 0.37,	FR: 39.74,	TDR: 0.99
Param Set: 96,	AP: 0.28,	AR: 0.78,	F1: 0.41,	FR: 55.50,	TDR: 0.92
Param Set: 54,	AP: 0.24,	AR: 0.87,	F1: 0.37,	FR: 40.95,	TDR: 0.94
Param Set: 98,	AP: 0.28,	AR: 0.79,	F1: 0.41,	FR: 54.01,	TDR: 0.92
Param Set: 37,	AP: 0.24,	AR: 0.88,	F1: 0.38,	FR: 40.88,	TDR: 0.98
Param Set: 39,	AP: 0.24,	AR: 0.87,	F1: 0.37,	FR: 41.35,	TDR: 0.94
Param Set: 55,	AP: 0.24,	AR: 0.89,	F1: 0.38,	FR: 40.48,	TDR: 0.99
Param Set: 97,	AP: 0.28,	AR: 0.79,	F1: 0.41,	FR: 54.65,	TDR: 0.92
Param Set: 90,	AP: 0.28,	AR: 0.78,	F1: 0.41,	FR: 55.50,	TDR: 0.92
Param Set: 79,	AP: 0.28,	AR: 0.79,	F1: 0.41,	FR: 54.80,	TDR: 0.92
Param Set: 41,	AP: 0.24,	AR: 0.88,	F1: 0.37,	FR: 40.08,	TDR: 0.98
Param Set: 77,	AP: 0.28,	AR: 0.80,	F1: 0.41,	FR: 54.11,	TDR: 0.92
Param Set: 24,	AP: 0.20,	AR: 0.94,	F1: 0.33,	FR: 23.33,	TDR: 0.99
Param Set: 23,	AP: 0.19,	AR: 0.94,	F1: 0.32,	FR: 22.21,	TDR: 0.99
Param Set: 4,	AP: 0.20,	AR: 0.94,	F1: 0.33,	FR: 23.58,	TDR: 0.99
Param Set: 3,	AP: 0.20,	AR: 0.94,	F1: 0.33,	FR: 24.37,	TDR: 0.99
Param Set: 76,	AP: 0.28,	AR: 0.79,	F1: 0.41,	FR: 54.80,	TDR: 0.92
Param Set: 40,	AP: 0.24,	AR: 0.88,	F1: 0.38,	FR: 40.88,	TDR: 0.98
Param Set: 78,	AP: 0.28,	AR: 0.78,	F1: 0.41,	FR: 55.75,	TDR: 0.92
Param Set: 2,	AP: 0.20,	AR: 0.94,	F1: 0.32,	FR: 23.03,	TDR: 0.99
Param Set: 5,	AP: 0.20,	AR: 0.94,	F1: 0.32,	FR: 23.03,	TDR: 0.99
Param Set: 22,	AP: 0.20,	AR: 0.94,	F1: 0.32,	FR: 22.61,	TDR: 0.99
Param Set: 25,	AP: 0.20,	AR: 0.94,	F1: 0.32,	FR: 22.61,	TDR: 0.99
```

## Test Set

#### Filters: [ 'mean_filter', 'turbine_filter', 'denoise_filter', 'intensity_filter', 'tracklet_association' ]

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.05,	TDR: 1.00
- 88.json: AP: 0.02,	AR: 0.62,	F1: 0.04,	FR: 70.37,	TDR: 0.98
- 75.json: AP: 0.02,	AR: 0.82,	F1: 0.03,	FR: 45.14,	TDR: 1.00

### Filters: ['mean_filter', 'turbine_filter', 'intensity_filter', 'tracklet_association']

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00
- 88.json: AP: 0.01,	AR: 0.99,	F1: 0.02,	FR: 5.30,	TDR: 1.00
- 75.json: AP: 0.01,	AR: 0.99,	F1: 0.02,	FR: 5.30,	TDR: 1.00

#### Filters: [ 'mean_filter', 'turbine_filter', 'denoise_filter', 'tracklet_association' ]

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00
- 88.json: AP: 0.01,	AR: 0.98,	F1: 0.02,	FR: 3.65,	TDR: 1.00
- 75.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 1.13,	TDR: 1.00

#### Filters: ['mean_filter', 'denoise_filter', 'intensity_filter', 'tracklet_association']

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.04,	TDR: 1.00
- 88.json: AP: 0.02,	AR: 0.62,	F1: 0.04,	FR: 70.26,	TDR: 0.98
- 75.json: AP: 0.02,	AR: 0.82,	F1: 0.03,	FR: 45.03,	TDR: 1.00

#### Filters: ['turbine_filter', 'denoise_filter', 'intensity_filter', 'tracklet_association']

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00
- 88.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.09,	TDR: 1.00
- 75.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00

#### Filters: [ 'mean_filter', 'turbine_filter', 'denoise_filter', 'intensity_filter' ]

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.05,	TDR: 1.00
- 88.json: AP: 0.02,	AR: 0.62,	F1: 0.04,	FR: 70.37,	TDR: 0.98
- 75.json: AP: 0.02,	AR: 0.82,	F1: 0.03,	FR: 45.14,	TDR: 1.00

#### Filters: [ 'mean_filter', 'turbine_filter', 'denoise_filter' ]

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00
- 88.json: AP: 0.01,	AR: 0.98,	F1: 0.02,	FR: 3.65,	TDR: 1.00
- 75.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 1.13,	TDR: 1.00

#### Filters: ['mean_filter', 'turbine_filter', 'intensity_filter'],

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00
- 88.json: AP: 0.01,	AR: 0.99,	F1: 0.02,	FR: 5.30,	TDR: 1.00
- 75.json: AP: 0.01,	AR: 0.99,	F1: 0.02,	FR: 5.30,	TDR: 1.00

#### Filters: ['mean_filter', 'denoise_filter', 'intensity_filter']

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.04,	TDR: 1.00
- 88.json: AP: 0.02,	AR: 0.62,	F1: 0.04,	FR: 70.26,	TDR: 0.98
- 75.json: AP: 0.02,	AR: 0.82,	F1: 0.03,	FR: 45.03,	TDR: 1.00

#### Filters: ['turbine_filter', 'denoise_filter', 'intensity_filter']

- 20.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00
- 88.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.09,	TDR: 1.00
- 75.json: AP: 0.01,	AR: 1.00,	F1: 0.02,	FR: 0.00,	TDR: 1.00